# 4 监督模型
本章开始将对sklean中主流的监督学习模型实现方法进行归纳总结。
## 4.1 线性模型
### 4.1.1 线性回归：普通最小二乘法
关注点1：**非奇异性**，奇异性的原因：

1. $X$ 存在线性相关关系(多重共线性), 即非满秩矩阵；其实变量相关就会产生影响。
2. 数据特征比数据量多，非满秩。

关注点2：中心化和标准化

In [1]:
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True,
                                    normalize=True,
                                    copy_X=True,
                                    n_jobs=None) # positive=True,强制将系数转换为非负数,0.24以上版本才支持
reg.fit([[0,0],[1,1],[2,2]],[0.2,-0.7,-1.9])

LinearRegression(normalize=True)

In [2]:
reg.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': True}

In [3]:
print('参数矩阵的秩：%s,\n参数矩阵的奇异值：%s,\n系数矩阵：%s,\n截距项：%s,\nR方：%s。' % (reg.rank_,reg.singular_,reg.coef_,reg.intercept_,reg.score([[0,0],[1,1],[2,2]],[0.2,-0.8,-1.8])))

参数矩阵的秩：1,
参数矩阵的奇异值：[1.41421356e+00 7.85046229e-17],
系数矩阵：[-0.525 -0.525],
截距项：0.2499999999999999,
R方：0.9975。


### 4.1.2 岭回归与分类
但是如果该矩阵是个奇异矩阵，则无法对其进行求解。那么什么情况下该矩阵会有奇异性呢?

X本身存在线性相关关系(多重共线性), 即非满秩矩阵。如果数据的特征中存在两个相关的变量，即使并不是完全线性相关，但是也会造成矩阵求逆的时候造成求解不稳定。
当数据特征比数据量还要多的时候, 即 [公式] , 这时候矩阵 [公式] 是一个矮胖型的矩阵，非满秩。


岭迹图
可以知道求得的岭系数 [公式] 是岭参数 [公式] 的函数，不同的 [公式] 得到不同的岭参数 [公式] , 因此我们可以增大 [公式] 的值来得到岭回归系数的变化，以及岭参数的变化轨迹图(岭迹图), 不存在奇异性时，岭迹图应稳定的逐渐趋向于0。

通过岭迹图我们可以:

观察较佳的 [公式] 取值
观察变量是否有多重共线性


#### 4.1.2.1 岭回归

In [5]:
from sklearn import linear_model # Ridge,RidgeClassifier,RidgeCV,RidgeClassifierCV
ridge = linear_model.Ridge(fit_intercept=True,
                           normalize=False,
                           copy_X=True,
                           alpha=1.0, # 浮点数或者与目标一致的数组，数值越大，惩罚越大
                           max_iter=None, # 共轭函数求解迭代次数
                           tol=0.001, # 预测精度
                           solver='auto', # auto,svd,cholesky,lsqr,sparse_cg,sag,saga
                           random_state=None) # 控制sag、saga


ridge.coef_
ridge.intercept_
ridge.n_iter_ # 仅sag and lsqr solvers

#### 4.1.2.2 岭分类
二分类：先将类别转换为 $-1,1$ 两种标签，然后使用回归的方式计算。

In [ ]:
from sklearn import linear_model
ridgeClassiflier = linear_model.RidgeClassifier(fit_intercept=True,
                                                normalize=False,
                                                copy_X=True,
                                                alpha=0.1,
                                                max_iter=None,
                                                slover='auto',
                                                tol=0.01,
                                                class_weight=None,# 目标权重，dict，balanced
                                                random_state=None)

In [ ]:
ridgeClassiflier.coef_
ridgeClassiflier.intercept_
ridgeClassiflier.classes_
ridgeClassiflier.n_iter_
ridgeClassiflier.decision_function() # 置信度

In [ ]:
from sklearn import RidgeCV,RidgeClassifierCV
RidgeCV(alpha=[0.1,1,10],
fit_intercept=,
normalize=,
scoring=, # 评估函数
cv=, # None：留一交叉验证，int：指定折叠数
gcv_mode=, # auto,svd,eigen
store_cv_values=True,# 仅cv=None才能使用
alpha_per_target=)

RidgeCV(alpha=[0.1,1,10],
fit_intercept=,
normalize=,
scoring=, # 评估函数
cv=, # None：留一交叉验证，int：指定折叠数
gcv_mode=, # auto,svd,eigen
store_cv_values=True,# 仅cv=None才能使用
class_weight=,
alpha_per_target=)

In [ ]:
cv_values_
coef_
intercept_
alpha_
best_score_
#classes_

In [ ]:
from sklearn import kernel_ridge
kernel_ridge.KernelRidge(alpha=,
kernel='linear', # linear,rbf,sigmoid,poly/polynomial,laplacian,cosine,chi2,additive_chi2
gamma=, # rbf,laplacian,poly,chi2,sigmoid 核中的参数，使用其他核时无效
degree=, # poly核中的参数d，使用其他核时无效。
coef0=, # poly和sigmoid核中的0参数的替代值，使用其他核时无效。
kernel_params=None)

### 4.1.3 Lasso

Lasso相比于普通最小二乘估计，可以在变量众多的时候快速有效地提取出重要变量，简化模型。

LASSO回归系数轨迹
类似岭轨迹，我们也可以改变λλ的值得到不同的回归系数，通过作图可以看到回归系数的轨迹

本文介绍了两种回归中的缩减方法，岭回归和LASSO。两种回归均是在标准线性回归的基础上加上正则项来减小模型的方差。这里其实便涉及到了权衡偏差(Bias)和方差(Variance)的问题。方差针对的是模型之间的差异，即不同的训练数据得到模型的区别越大说明模型的方差越大。而偏差指的是模型预测值与样本数据之间的差异。所以为了在过拟合和欠拟合之前进行权衡，我们需要确定适当的模型复杂度来使得总误差最小。


In [ ]:
## 4.2 线性和二次判别分析
## 4.3 内核岭回归
## 4.4 支持向量机
## 4.5 随机梯度下降法
## 4.6 最近的邻居
## 4.7 高斯过程
## 4.8 横向分解
## 4.9 朴素贝叶斯
## 4.10 决策树
## 4.11 整体方法
## 4.12 多类和多输出算法
## 4.13 特征选择
## 4.14 Semi-supervised学习
## 4.15 等张回归
## 4.16 概率校准
## 4.17 神经网络模型(监督)

## 过拟合